In [ ]:
import cv2, time, os, json
import numpy as np
import pandas as pd
from pathlib import Path

# --- 사용자 설정 ---
video_path = Path(r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250901_영상\2try\영상분석\Flat분석.mp4")
side_length_mm = 66.0

# --- 저장 프레임 간격 설정 ---
SAVE_EVERY = 5
SAVE_OFFSET = 0
CSV_ONLY_SAVED = False
EXCLUDE_FRAME0 = True
DEBUG_PRINT_SAVED = True

# --- 경로/유틸 ---
base_folder   = video_path.parent / "video_analysis"
Original_folder = base_folder / "Original"
Binary_folder   = base_folder / "Binary"
csv_path      = base_folder / f"{video_path.stem}_area.csv"
params_path   = base_folder / "last_params.json"

base_folder.mkdir(parents=True, exist_ok=True)

def odd_from_slider(x: int) -> int:
    if x <= 1: return 0
    v = x if x % 2 == 1 else x - 1
    return max(3, v)

def win_longpath(path: Path) -> str:
    s = str(path)
    if os.name == "nt":
        if len(s) >= 2 and s[1] == ":" and not s.startswith("\\\\?\\"):
            return "\\\\?\\" + s
    return s

def safe_write_image(path: Path, img) -> bool:
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"[ERR] make dir failed: {e}")
        return False
    ok = cv2.imwrite(win_longpath(path), img)
    if ok:
        return True
    try:
        ext = (path.suffix or ".png").lower()
        if ext not in [".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".tif", ".webp"]:
            ext = ".png"
        ret, buf = cv2.imencode(ext, img)
        if ret:
            buf.tofile(win_longpath(path))
            return True
        else:
            print(f"[ERR] imencode failed for: {path}")
    except Exception as e:
        print(f"[ERR] fallback write failed for {path}: {e}")
    return False

# --- 마스크 생성(색 임계 + 모폴로지) ---
def preprocess_and_mask(Original_bgr, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K):
    img = Original_bgr
    if BLUR_K >= 3:
        img = cv2.medianBlur(img, BLUR_K)

    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    R = rgb[:, :, 0]
    G = rgb[:, :, 1]
    B = rgb[:, :, 2]

    cond = (R >= R_MIN) & (G >= G_MIN) & (B >= B_MIN)
    mask = np.zeros(G.shape, dtype=np.uint8)
    mask[cond] = 255

    if MORPH_K >= 3:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (MORPH_K, MORPH_K))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=1)
    return mask

# --- LargestOnly 적용 ---
def keep_largest(mask):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num_labels > 1:
        largest = 1 + int(np.argmax(stats[1:, cv2.CC_STAT_AREA]))
        return np.where(labels == largest, 255, 0).astype(np.uint8)
    else:
        return np.zeros_like(mask)

# --- 오버레이 ---
def overlay_mask(Original_bgr, mask, alpha=0.45):
    color = np.zeros_like(Original_bgr); color[:] = (0, 255, 0)
    mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    return np.where(mask3 > 0,
                    ((1 - alpha) * Original_bgr + alpha * color).astype(np.uint8),
                    Original_bgr)

def put_multiline(img, lines, org=(10, 25), scale=1.0, color=(0, 255, 255),
                  thickness=1, line_gap=30):
    x, y = org
    for line in lines:
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, color, thickness, cv2.LINE_AA)
        y += line_gap
    return img

# ---- 파라미터 저장/복원 ----
DEFAULT_PARAMS = {
    "R_MIN": 160, "G_MIN": 160, "B_MIN": 140,
    "BLUR_K_RAW": 0, "MORPH_K_RAW": 0,
    "LargestOnly": 1,
    "InvertMask": 1,
    "SideCutPercent": 25,

    # 시간 차분 파라미터
    "UseTemporal": 1,     # 0/1
    "Warmup": 30,         # 배경 학습 프레임 수
    "Diff_T": 12,         # 회색 차분 임계
    "MOG2Var": 16,        # MOG2 varThreshold
    "HoldMask": 1         # 움직임 멈춰도 마지막 마스크 유지
}
def load_params():
    try:
        with open(win_longpath(params_path), "r", encoding="utf-8") as f:
            d = json.load(f)
        return {**DEFAULT_PARAMS, **d}
    except Exception:
        return DEFAULT_PARAMS.copy()

def save_params(d):
    try:
        params_path.parent.mkdir(parents=True, exist_ok=True)
        tmp = params_path.with_suffix(".json.tmp")
        with open(win_longpath(tmp), "w", encoding="utf-8") as f:
            json.dump(d, f, ensure_ascii=False, indent=2)
        os.replace(win_longpath(tmp), win_longpath(params_path))
    except Exception as e:
        print(f"[WARN] 파라미터 저장 실패: {e}")

# ========= 비디오/스케일 =========
cap = cv2.VideoCapture(str(video_path))
if not cap.isOpened():
    raise FileNotFoundError(f"영상 열기 실패: {video_path}")
ret, first = cap.read()
if not ret:
    raise RuntimeError("첫 프레임 읽기 실패")
H, W = first.shape[:2]
mm_per_px = side_length_mm / W
area_mm2_per_px = mm_per_px ** 2
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()
print(f"[INFO] 해상도 {W}x{H}, 총 프레임 {total_frames}, 1px = {mm_per_px:.5f} mm, 1px² = {area_mm2_per_px:.6f} mm²")

# ========= 트랙바 =========
cv2.namedWindow("Preview", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Preview", min(1500, W * 3), min(900, H * 2))

def _nothing(_): pass
cv2.createTrackbar("R_MIN",       "Preview", 0, 255, _nothing)
cv2.createTrackbar("G_MIN",       "Preview", 0, 255, _nothing)
cv2.createTrackbar("B_MIN",       "Preview", 0, 255, _nothing)
cv2.createTrackbar("BLUR_K",      "Preview", 0, 31, _nothing)
cv2.createTrackbar("MORPH_K",     "Preview", 0, 31, _nothing)
cv2.createTrackbar("LargestOnly", "Preview", 0, 1, _nothing)
cv2.createTrackbar("InvertMask",  "Preview", 0, 1, _nothing)
cv2.createTrackbar("SideCutPercent", "Preview", 0, 50, _nothing)

# 시간 차분 관련
cv2.createTrackbar("UseTemporal", "Preview", 0, 1, _nothing)
cv2.createTrackbar("Warmup",      "Preview", 0, 200, _nothing)
cv2.createTrackbar("Diff_T",      "Preview", 0, 100, _nothing)
cv2.createTrackbar("MOG2Var",     "Preview", 0, 50, _nothing)
cv2.createTrackbar("HoldMask",    "Preview", 0, 1, _nothing)

# ---- 저장된 값으로 초기화 ----
p = load_params()
cv2.setTrackbarPos("R_MIN",       "Preview", int(p["R_MIN"]))
cv2.setTrackbarPos("G_MIN",       "Preview", int(p["G_MIN"]))
cv2.setTrackbarPos("B_MIN",       "Preview", int(p["B_MIN"]))
cv2.setTrackbarPos("BLUR_K",      "Preview", int(p["BLUR_K_RAW"]))
cv2.setTrackbarPos("MORPH_K",     "Preview", int(p["MORPH_K_RAW"]))
cv2.setTrackbarPos("LargestOnly", "Preview", int(p["LargestOnly"]))
cv2.setTrackbarPos("InvertMask",  "Preview", int(p["InvertMask"]))
cv2.setTrackbarPos("SideCutPercent", "Preview", int(p["SideCutPercent"]))
cv2.setTrackbarPos("UseTemporal", "Preview", int(p["UseTemporal"]))
cv2.setTrackbarPos("Warmup",      "Preview", int(p["Warmup"]))
cv2.setTrackbarPos("Diff_T",      "Preview", int(p["Diff_T"]))
cv2.setTrackbarPos("MOG2Var",     "Preview", int(p["MOG2Var"]))
cv2.setTrackbarPos("HoldMask",    "Preview", int(p["HoldMask"]))

# ========= 시간 차분 보조 =========
def create_bg_sub(var_thresh: int, history: int = 500):
    var_thresh = max(1, int(var_thresh))
    return cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=var_thresh, detectShadows=False)

bg_sub = create_bg_sub(int(p["MOG2Var"]))
warmup_count = 0
last_good_mask = None
prev_use_temporal = int(p["UseTemporal"])
prev_mog2var     = int(p["MOG2Var"])

# ========= 공통 파이프라인 =========
def build_mask(frame_bgr, params, state_dict=None):
    """
    params: dict (슬라이더에서 읽은 값)
    state_dict: dict with keys: bg_sub, warmup_count, last_good_mask
    """
    R_MIN   = params["R_MIN"]
    G_MIN   = params["G_MIN"]
    B_MIN   = params["B_MIN"]
    BLUR_K  = odd_from_slider(params["BLUR_K_RAW"])
    MORPH_K = odd_from_slider(params["MORPH_K_RAW"])
    use_largest = bool(params["LargestOnly"])
    invert_flag = bool(params["InvertMask"])
    cut_percent = params["SideCutPercent"] / 100.0

    # 1) 색 임계 마스크
    color_mask = preprocess_and_mask(frame_bgr, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K)

    # 2) 시간 차분 마스크
    final_mask = color_mask.copy()
    if params["UseTemporal"] and state_dict is not None:
        # MOG2Var 변경 시 즉시 재생성
        if params["MOG2Var"] != state_dict.get("prev_mog2var", params["MOG2Var"]):
            state_dict["bg_sub"] = create_bg_sub(params["MOG2Var"])
            state_dict["warmup_count"] = 0
            state_dict["last_good_mask"] = None
            state_dict["prev_mog2var"] = params["MOG2Var"]

        # Warmup/UseTemporal 토글 시 warmup 초기화
        if params["UseTemporal"] != state_dict.get("prev_use_temporal", params["UseTemporal"]):
            state_dict["warmup_count"] = 0
            state_dict["last_good_mask"] = None
            state_dict["prev_use_temporal"] = params["UseTemporal"]

        # 적용
        frame_blur = frame_bgr if BLUR_K < 3 else cv2.medianBlur(frame_bgr, BLUR_K)
        fg = state_dict["bg_sub"].apply(frame_blur)  # 0~255
        state_dict["warmup_count"] += 1

        fg_mask = np.zeros_like(fg)
        if state_dict["warmup_count"] >= params["Warmup"]:
            # 회색 차분 보강
            bg_img = state_dict["bg_sub"].getBackgroundImage()
            if bg_img is not None:
                gray = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY)
                bg_gray = cv2.cvtColor(bg_img, cv2.COLOR_BGR2GRAY)
                diff = cv2.absdiff(gray, bg_gray)
                _, fg2 = cv2.threshold(diff, int(params["Diff_T"]), 255, cv2.THRESH_BINARY)
                fg = cv2.bitwise_or(fg, fg2)

            if MORPH_K >= 3:
                k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (MORPH_K, MORPH_K))
                fg = cv2.morphologyEx(fg, cv2.MORPH_OPEN, k, iterations=1)
                fg = cv2.morphologyEx(fg, cv2.MORPH_CLOSE, k, iterations=1)

            fg_mask = fg

        # OR 결합
        final_mask = cv2.bitwise_or(final_mask, fg_mask)

        # HoldMask: 비어 있으면 마지막 마스크 유지
        if params["HoldMask"]:
            if np.count_nonzero(final_mask) > 0:
                state_dict["last_good_mask"] = final_mask.copy()
            else:
                if state_dict["last_good_mask"] is not None:
                    final_mask = state_dict["last_good_mask"].copy()

    # 3) 사이드 컷
    Hh, Ww = final_mask.shape
    left_cut  = int(Ww * cut_percent)
    right_cut = int(Ww * (1 - cut_percent))
    region = np.zeros_like(final_mask)
    region[:, left_cut:right_cut] = 255
    final_mask = cv2.bitwise_and(final_mask, region)

    # 4) LargestOnly (최종 단계에서 적용)
    if use_largest:
        final_mask = keep_largest(final_mask)

    # 5) 반전
    if invert_flag:
        final_mask = cv2.bitwise_not(final_mask)

    return final_mask

# ========= 저장 루틴 =========
def save_all(params):
    print("[INFO] 저장 시작...")
    base_folder.mkdir(parents=True, exist_ok=True)
    Original_folder.mkdir(parents=True, exist_ok=True)
    Binary_folder.mkdir(parents=True, exist_ok=True)

    cap2 = cv2.VideoCapture(str(video_path))
    if not cap2.isOpened():
        print("[ERR] 저장용 VideoCapture 열기 실패")
        return

    # 저장용 독립 BG 모델 (프리뷰와 분리)
    state = {
        "bg_sub": create_bg_sub(int(params["MOG2Var"])),
        "warmup_count": 0,
        "last_good_mask": None,
        "prev_mog2var": int(params["MOG2Var"]),
        "prev_use_temporal": int(params["UseTemporal"])
    }

    frame_list, area_px_list, area_mm2_list = [], [], []
    j = 0
    write_fail = 0
    saved_count = 0

    every_n = max(1, int(SAVE_EVERY))
    offset_n = int(SAVE_OFFSET) % every_n

    while True:
        ret2, fr2 = cap2.read()
        if not ret2:
            break

        m2 = build_mask(fr2, params, state)

        px = int(np.count_nonzero(m2))
        is_target = ((j - offset_n) % every_n == 0)
        if EXCLUDE_FRAME0 and j == 0:
            is_target = False

        if not CSV_ONLY_SAVED or is_target:
            frame_list.append(j)
            area_px_list.append(px)
            area_mm2_list.append(px * area_mm2_per_px)

        if is_target:
            f_name = f"Original_{j:05d}.png"
            m_name = f"Binary_{j:05d}.png"
            ok1 = safe_write_image(Original_folder / f_name, fr2)
            ok2 = safe_write_image(Binary_folder / m_name, m2)
            if not ok1 or not ok2:
                write_fail += 1
                if not ok1: print(f"[WARN] 원본 저장 실패: {Original_folder / f_name}")
                if not ok2: print(f"[WARN] 마스크 저장 실패: {Binary_folder / m_name}")
            else:
                saved_count += 1
                if DEBUG_PRINT_SAVED:
                    print(f"[SAVE] frame {j} saved (every={every_n}, offset={offset_n})")

        if j % 100 == 0:
            print(f"  processed frame {j}")
        j += 1

    cap2.release()

    try:
        df = pd.DataFrame({
            "frame": frame_list,
            "area_px": area_px_list,
            "area_mm2": area_mm2_list
        })
        csv_tmp = base_folder / f"{video_path.stem}_area_tmp.csv"
        df.to_csv(win_longpath(csv_tmp), index=False, encoding="utf-8-sig")
        os.replace(win_longpath(csv_tmp), win_longpath(csv_path))
        print(f"[INFO] CSV 저장 완료: {csv_path}")
    except Exception as e:
        print(f"[ERR] CSV 저장 실패: {e}")

    print(f"[INFO] 저장 완료:\n"
          f" - Original: {Original_folder} (saved {saved_count} files)\n"
          f" - Binary  : {Binary_folder} (saved {saved_count} files)\n"
          f" - CSV rows: {len(frame_list)} "
          f"({'only saved frames' if CSV_ONLY_SAVED else 'all frames'})\n"
          f" - Interval: every {every_n} frames (offset {offset_n})\n"
          f" - write_fail: {write_fail}")

# ========= 프리뷰 루프 =========
idx = 0; paused = False

def read_slider_raw():
    return {
        "R_MIN": cv2.getTrackbarPos("R_MIN", "Preview"),
        "G_MIN": cv2.getTrackbarPos("G_MIN", "Preview"),
        "B_MIN": cv2.getTrackbarPos("B_MIN", "Preview"),
        "BLUR_K_RAW": cv2.getTrackbarPos("BLUR_K", "Preview"),
        "MORPH_K_RAW": cv2.getTrackbarPos("MORPH_K", "Preview"),
        "LargestOnly": cv2.getTrackbarPos("LargestOnly", "Preview"),
        "InvertMask": cv2.getTrackbarPos("InvertMask", "Preview"),
        "SideCutPercent": cv2.getTrackbarPos("SideCutPercent", "Preview"),

        "UseTemporal": cv2.getTrackbarPos("UseTemporal", "Preview"),
        "Warmup":      cv2.getTrackbarPos("Warmup", "Preview"),
        "Diff_T":      cv2.getTrackbarPos("Diff_T", "Preview"),
        "MOG2Var":     cv2.getTrackbarPos("MOG2Var", "Preview"),
        "HoldMask":    cv2.getTrackbarPos("HoldMask", "Preview"),
    }

try:
    while True:
        idx = idx % max(1, total_frames)
        cap = cv2.VideoCapture(str(video_path))
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read(); cap.release()
        if not ret:
            idx += 1
            continue

        raw = read_slider_raw()

        # 프리뷰용 상태 업데이트(모델 재생성/워밍업 리셋)
        if raw["MOG2Var"] != prev_mog2var:
            bg_sub = create_bg_sub(int(raw["MOG2Var"]))
            warmup_count = 0
            last_good_mask = None
            prev_mog2var = raw["MOG2Var"]
        if raw["UseTemporal"] != prev_use_temporal:
            warmup_count = 0
            last_good_mask = None
            prev_use_temporal = raw["UseTemporal"]

        # state dict (프리뷰 전용)
        state_preview = {
            "bg_sub": bg_sub,
            "warmup_count": warmup_count,
            "last_good_mask": last_good_mask,
            "prev_mog2var": prev_mog2var,
            "prev_use_temporal": prev_use_temporal
        }

        mask = build_mask(frame, {
            **raw,
            # build_mask 내부에서 odd_from_slider로 변환하므로 RAW 값 그대로 전달
        }, state_preview)

        # state 업데이트(워밍업 카운터/마지막 마스크 유지)
        warmup_count = state_preview["warmup_count"]
        last_good_mask = state_preview["last_good_mask"]

        overlay = overlay_mask(frame, mask, alpha=0.45)
        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        panel = np.hstack([frame, overlay, mask_bgr])

        lines = [
            f"Frame {idx}/{total_frames-1}",
            "Controls: SPACE=Pause/Play, S=Save(Original+Binary+CSV), R=Restart, Q=Quit",
            f"Save interval: every {SAVE_EVERY} frames (offset {SAVE_OFFSET}), CSV={'saved only' if CSV_ONLY_SAVED else 'all frames'}",
            f"Params: R={raw['R_MIN']} G={raw['G_MIN']} B={raw['B_MIN']}  BLUR={odd_from_slider(raw['BLUR_K_RAW'])}  MORPH={odd_from_slider(raw['MORPH_K_RAW'])}  LargestOnly={raw['LargestOnly']}  Invert={raw['InvertMask']}  Cut={raw['SideCutPercent']}%",
            f"Temporal: Use={raw['UseTemporal']}  Warmup={raw['Warmup']}  Diff_T={raw['Diff_T']}  MOG2Var={raw['MOG2Var']}  Hold={raw['HoldMask']}  Warmed={warmup_count}",
        ]
        panel = put_multiline(panel, lines, org=(10, 30))
        cv2.imshow("Preview", panel)

        key = cv2.waitKey(1 if not paused else 30) & 0xFF
        if key == ord('q'):
            save_params(raw)  # 종료 직전, 마지막 값 저장
            break
        elif key == 32:  # SPACE
            paused = not paused
        elif key == ord('s'):
            print("[INFO] 's' 감지 → 저장을 시작합니다. (Preview 창 포커스 필요)")
            save_all(raw)
            save_params(raw)  # 저장 시점에도 현재 민감도 저장
        elif key == ord('r'):
            print("[INFO] 'r' 감지 → 영상 처음부터 재시작")
            idx = 0
            paused = False
            # 배경 재학습
            bg_sub = create_bg_sub(int(raw["MOG2Var"]))
            warmup_count = 0
            last_good_mask = None

        if not paused:
            idx += 1
        else:
            time.sleep(0.02)
finally:
    cv2.destroyAllWindows()
    print("[INFO] 종료")


총 프레임 수: 300
FPS: 30.0
동영상 길이: 10.000 초
